In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pandas import DataFrame, read_csv
import json

import sys
sys.path.insert(0, '../ToolBox')

import CrossValidation, MLP, Helper, ZScore

import os, sys
import cPickle

In [ ]:
saved_train_data = open('pre_processed_data/train_preprocessed.pkl', 'rb')
train_data_loaded = cPickle.load(saved_train_data)

In [ ]:
saved_test_data = open('pre_processed_data/test_preprocessed.pkl', 'rb')
test_data_loaded = cPickle.load(saved_test_data)

In [ ]:
import yaml

with open("../ToolBox/params.json") as data_file:
    params = yaml.safe_load(data_file)

dict_zscore = ZScore.init()

print "Starting normalization of train data."
train_mean, train_stddev = dict_zscore["z_score_compute"](train_data_loaded)
print "Mean and stddev calculated."
normalized_train_data = dict_zscore["z_score_normalize"](train_data_loaded, train_mean, train_stddev)
print "Normalized."

print np.array(normalized_train_data).shape

data_dict={"y": Helper.convert_one_hot_encoding(np.subtract(np.array(labels), 1), params['MLP']["output-dim"]), "x":np.array(normalized_train_data)}

print data_dict["y"].shape
print params['MLP']["output-dim"]

print "Starting training."
dict_mlp = MLP.init(params['MLP'])
print "Training."
dict_mlp['train'](data_dict)
print "Training finished."

print "Starting normalization of test data."
normalized_test_data = dict_zscore["z_score_normalize"](test_data_loaded, train_mean, train_stddev)
print "Predicting with test data."
print normalized_test_data.shape
result_dict = dict_mlp['predict'](normalized_test_data)
print "Prediction:"
result_dict

In [ ]:
lis = []
lis.append("ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer")

for idx, z in enumerate(result_dict["network_0"]):
    zList = z[0].tolist()
    temp_str = str(idx+1)
    for idy, x in enumerate(zList):
        temp_str = temp_str+", "+str(x)
    lis.append(temp_str)


lis = np.array(lis)
np.savetxt('result_shelter_animal_outcomes.csv', lis, fmt='%s', delimiter=',')